In [87]:
#importing data management, data visualisation libraries.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import regex as re

In [88]:
data= pd.read_csv('KL_no_null_no_duplicates_data.csv')
data

,Location,Price,Rooms,Bathrooms,Car Parks,Property Type,Size,Furnishing
0,"KLCC, Kuala Lumpur","RM 1,250,000",2+1,3,2,Serviced Residence,"Built-up : 1,335 sq. ft.",Fully Furnished
1,"Dutamas, Kuala Lumpur","RM 1,030,000",3,4,2,Condominium (Corner),"Built-up : 1,875 sq. ft.",Partly Furnished
2,"Bukit Jalil, Kuala Lumpur","RM 900,000",4+1,3,2,Condominium (Corner),"Built-up : 1,513 sq. ft.",Partly Furnished
3,"Taman Tun Dr Ismail, Kuala Lumpur","RM 5,350,000",4+2,5,4,Bungalow,Land area : 7200 sq. ft.,Partly Furnished
4,"Taman Tun Dr Ismail, Kuala Lumpur","RM 2,600,000",5,4,4,Semi-detached House,Land area : 3600 sq. ft.,Partly Furnished
...,...,...,...,...,...,...,...,...
29027,"Seputeh, Kuala Lumpur","RM 750,000",3,2,1,Condominium,Built-up : 915 sq. ft.,Partly Furnished
29028,"KL Sentral, Kuala Lumpur","RM 1,400,000",3+1,3,2,Condominium (Corner),Land area : 1544 sq. ft.,Fully Furnished
29029,"KL Eco City, Kuala Lumpur","RM 880,000",1,1,1,Condominium (Corner),Built-up : 650 sq. ft.,Partly Furnished
29030,"Sri Hartamas, Kuala Lumpur","RM 2,700,000",4+2,6,3,Condominium (Corner),"Built-up : 3,973 sq. ft.",Partly Furnished


In [89]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29032 entries, 0 to 29031
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Location       29032 non-null  object
 1   Price          29032 non-null  object
 2   Rooms          29032 non-null  object
 3   Bathrooms      29032 non-null  int64 
 4   Car Parks      29032 non-null  int64 
 5   Property Type  29032 non-null  object
 6   Size           29032 non-null  object
 7   Furnishing     29032 non-null  object
dtypes: int64(2), object(6)
memory usage: 1.8+ MB


In [90]:
# Function to extract type, value, and unit (Cleaning size column)
def extract_size_info(row):
   # Regular expression pattern to match type, value, and unit
   pattern = r'(?P<type>.+?)\s*:\s*(?P<value>[\d,\/xX]+)\s*(?P<unit>[a-zA-Z.]+)'
   match = re.match(pattern, row)
   if match:
       value = match.group('value').replace(',', '')  # Remove commas from value
       if any(op in value for op in ('x', 'X', '/')):  # If there is '*' or '/' indicating multiplication
           try:
               value = eval(value)  # Evaluate the expression
           except SyntaxError:
               pass  # Skip evaluation for invalid expressions
       return match.group('type').strip(), str(value), match.group('unit').strip()
   else:
       return None, None, None
# Apply the function to split the size column
data[['type', 'size', 'unit']] = data['Size'].apply(extract_size_info).apply(pd.Series)
# Drop the original 'Size' column if needed
data.drop(columns=['Size'], inplace=True)
print(data)

                                Location         Price Rooms  Bathrooms  \
0                     KLCC, Kuala Lumpur  RM 1,250,000   2+1          3   
1                  Dutamas, Kuala Lumpur  RM 1,030,000     3          4   
2              Bukit Jalil, Kuala Lumpur    RM 900,000   4+1          3   
3      Taman Tun Dr Ismail, Kuala Lumpur  RM 5,350,000   4+2          5   
4      Taman Tun Dr Ismail, Kuala Lumpur  RM 2,600,000     5          4   
...                                  ...           ...   ...        ...   
29027              Seputeh, Kuala Lumpur    RM 750,000     3          2   
29028           KL Sentral, Kuala Lumpur  RM 1,400,000   3+1          3   
29029          KL Eco City, Kuala Lumpur    RM 880,000     1          1   
29030         Sri Hartamas, Kuala Lumpur  RM 2,700,000   4+2          6   
29031              Setapak, Kuala Lumpur    RM 540,000     4          2   

       Car Parks               Property Type        Furnishing       type  \
0              2      

In [91]:
# Remove non-numeric characters from the 'size' column
data['size'] = data['size'].str.replace('[^\d]+', '', regex=True)

# Convert the cleaned 'size' column to integers
# Convert the cleaned 'size' column to floating-point numbers
data['size'] = data['size'].astype('float')



In [92]:
data['size']

0        1335.0
1        1875.0
2        1513.0
3        7200.0
4        3600.0
          ...  
29027     915.0
29028    1544.0
29029     650.0
29030    3973.0
29031    1313.0
Name: size, Length: 29032, dtype: float64

In [93]:
data['size'].value_counts()

size
1100.0    392
950.0     294
2275.0    289
1200.0    265
850.0     236
         ... 
5079.0      1
2389.0      1
3709.0      1
5423.0      1
2132.0      1
Name: count, Length: 3109, dtype: int64

In [94]:
data['type'].value_counts()

type
Built-up     22604
Land area     6324
Name: count, dtype: int64

In [95]:
data['unit'].value_counts()

unit
sq.      28659
x          181
X           43
.           19
sf          12
acres        7
acre         4
ft           1
or           1
wt           1
Name: count, dtype: int64

# Function to extract type, value, and unit( Cleaning size column)
def extract_size_info(row):
    # Regular expression pattern to match type, value, and unit
    pattern = r'(?P<type>.+?)\s*:\s*(?P<value>[\d*,/]+)\s*(?P<unit>[a-zA-Z.]+)'
    match = re.match(pattern, row['Size'])
    if match:
        value = match.group('value').replace(',', '')  # Remove commas from value
        if '*' in value:  # If there is '*' indicating multiplication
            value = eval(value)  # Evaluate the expression
        return match.group('type').strip(), str(value), match.group('unit').strip()
    else:
        return None, None, None

# Apply the function to split the size column
data[['type', 'size', 'unit']] = data.apply(extract_size_info, axis=1, result_type='expand')

# Drop the original 'size' column if needed
data.drop(columns=['Size'], inplace=True)

data



In [96]:
data[['type', 'size', 'unit']].isna().sum()

type    104
size    104
unit    104
dtype: int64

In [97]:
data.dropna(inplace=True)

data.reset_index(drop=True, inplace=True) 

In [98]:
data[['type', 'size', 'unit']].isna().sum()

type    0
size    0
unit    0
dtype: int64

In [99]:
data['type'].value_counts()

type
Built-up     22604
Land area     6324
Name: count, dtype: int64

In [100]:
data['unit'].value_counts()

unit
sq.      28659
x          181
X           43
.           19
sf          12
acres        7
acre         4
ft           1
or           1
wt           1
Name: count, dtype: int64

In [101]:
data['size'].value_counts()

size
1100.0    392
950.0     294
2275.0    289
1200.0    265
850.0     236
         ... 
5079.0      1
2389.0      1
3709.0      1
5423.0      1
2132.0      1
Name: count, Length: 3109, dtype: int64

In [102]:
data[['type', 'size', 'unit']].value_counts()

type       size          unit
Built-up   1.100000e+03  sq.     351
Land area  2.275000e+03  sq.     289
Built-up   9.500000e+02  sq.     272
           1.200000e+03  sq.     231
           8.500000e+02  sq.     228
                                ... 
Land area  3.400000e+01  x         1
           3.200000e+01  X         1
           2.800000e+01  x         1
                         .         1
           3.466667e+15  sq.       1
Name: count, Length: 3991, dtype: int64

In [103]:
data

,Location,Price,Rooms,Bathrooms,Car Parks,Property Type,Furnishing,type,size,unit
0,"KLCC, Kuala Lumpur","RM 1,250,000",2+1,3,2,Serviced Residence,Fully Furnished,Built-up,1335.0,sq.
1,"Dutamas, Kuala Lumpur","RM 1,030,000",3,4,2,Condominium (Corner),Partly Furnished,Built-up,1875.0,sq.
2,"Bukit Jalil, Kuala Lumpur","RM 900,000",4+1,3,2,Condominium (Corner),Partly Furnished,Built-up,1513.0,sq.
3,"Taman Tun Dr Ismail, Kuala Lumpur","RM 5,350,000",4+2,5,4,Bungalow,Partly Furnished,Land area,7200.0,sq.
4,"Taman Tun Dr Ismail, Kuala Lumpur","RM 2,600,000",5,4,4,Semi-detached House,Partly Furnished,Land area,3600.0,sq.
...,...,...,...,...,...,...,...,...,...,...
28923,"Seputeh, Kuala Lumpur","RM 750,000",3,2,1,Condominium,Partly Furnished,Built-up,915.0,sq.
28924,"KL Sentral, Kuala Lumpur","RM 1,400,000",3+1,3,2,Condominium (Corner),Fully Furnished,Land area,1544.0,sq.
28925,"KL Eco City, Kuala Lumpur","RM 880,000",1,1,1,Condominium (Corner),Partly Furnished,Built-up,650.0,sq.
28926,"Sri Hartamas, Kuala Lumpur","RM 2,700,000",4+2,6,3,Condominium (Corner),Partly Furnished,Built-up,3973.0,sq.


In [104]:
#Cleaning Location column
# Split the column with commas into two separate columns
data[['area', 'L2', 'L3']] = data['Location'].str.split(',' ,expand=True)
data.drop(columns=["Location","L2","L3"], inplace=True)
data

,Price,Rooms,Bathrooms,Car Parks,Property Type,Furnishing,type,size,unit,area
0,"RM 1,250,000",2+1,3,2,Serviced Residence,Fully Furnished,Built-up,1335.0,sq.,KLCC
1,"RM 1,030,000",3,4,2,Condominium (Corner),Partly Furnished,Built-up,1875.0,sq.,Dutamas
2,"RM 900,000",4+1,3,2,Condominium (Corner),Partly Furnished,Built-up,1513.0,sq.,Bukit Jalil
3,"RM 5,350,000",4+2,5,4,Bungalow,Partly Furnished,Land area,7200.0,sq.,Taman Tun Dr Ismail
4,"RM 2,600,000",5,4,4,Semi-detached House,Partly Furnished,Land area,3600.0,sq.,Taman Tun Dr Ismail
...,...,...,...,...,...,...,...,...,...,...
28923,"RM 750,000",3,2,1,Condominium,Partly Furnished,Built-up,915.0,sq.,Seputeh
28924,"RM 1,400,000",3+1,3,2,Condominium (Corner),Fully Furnished,Land area,1544.0,sq.,KL Sentral
28925,"RM 880,000",1,1,1,Condominium (Corner),Partly Furnished,Built-up,650.0,sq.,KL Eco City
28926,"RM 2,700,000",4+2,6,3,Condominium (Corner),Partly Furnished,Built-up,3973.0,sq.,Sri Hartamas


In [105]:
#Cleaning Price column
data['Price'] = data['Price'].str.replace(r'\D+', '', regex=True).astype(int)

data

,Price,Rooms,Bathrooms,Car Parks,Property Type,Furnishing,type,size,unit,area
0,1250000,2+1,3,2,Serviced Residence,Fully Furnished,Built-up,1335.0,sq.,KLCC
1,1030000,3,4,2,Condominium (Corner),Partly Furnished,Built-up,1875.0,sq.,Dutamas
2,900000,4+1,3,2,Condominium (Corner),Partly Furnished,Built-up,1513.0,sq.,Bukit Jalil
3,5350000,4+2,5,4,Bungalow,Partly Furnished,Land area,7200.0,sq.,Taman Tun Dr Ismail
4,2600000,5,4,4,Semi-detached House,Partly Furnished,Land area,3600.0,sq.,Taman Tun Dr Ismail
...,...,...,...,...,...,...,...,...,...,...
28923,750000,3,2,1,Condominium,Partly Furnished,Built-up,915.0,sq.,Seputeh
28924,1400000,3+1,3,2,Condominium (Corner),Fully Furnished,Land area,1544.0,sq.,KL Sentral
28925,880000,1,1,1,Condominium (Corner),Partly Furnished,Built-up,650.0,sq.,KL Eco City
28926,2700000,4+2,6,3,Condominium (Corner),Partly Furnished,Built-up,3973.0,sq.,Sri Hartamas


In [106]:
# Splitting the Rooms column and creating two new columns, no of bed room, servant rooms
data[['No_of_Bedrooms', 'No_of_servant_rooms']] = data['Rooms'].str.split('+', expand=True)
data.drop(columns="Rooms", inplace=True)


In [107]:
data['No_of_servant_rooms'].isna().sum()

17188

In [108]:
data['No_of_servant_rooms'].fillna(value=0, inplace=True)

In [109]:
data

,Price,Bathrooms,Car Parks,Property Type,Furnishing,type,size,unit,area,No_of_Bedrooms,No_of_servant_rooms
0,1250000,3,2,Serviced Residence,Fully Furnished,Built-up,1335.0,sq.,KLCC,2,1
1,1030000,4,2,Condominium (Corner),Partly Furnished,Built-up,1875.0,sq.,Dutamas,3,0
2,900000,3,2,Condominium (Corner),Partly Furnished,Built-up,1513.0,sq.,Bukit Jalil,4,1
3,5350000,5,4,Bungalow,Partly Furnished,Land area,7200.0,sq.,Taman Tun Dr Ismail,4,2
4,2600000,4,4,Semi-detached House,Partly Furnished,Land area,3600.0,sq.,Taman Tun Dr Ismail,5,0
...,...,...,...,...,...,...,...,...,...,...,...
28923,750000,2,1,Condominium,Partly Furnished,Built-up,915.0,sq.,Seputeh,3,0
28924,1400000,3,2,Condominium (Corner),Fully Furnished,Land area,1544.0,sq.,KL Sentral,3,1
28925,880000,1,1,Condominium (Corner),Partly Furnished,Built-up,650.0,sq.,KL Eco City,1,0
28926,2700000,6,3,Condominium (Corner),Partly Furnished,Built-up,3973.0,sq.,Sri Hartamas,4,2


In [110]:
data['size'].isna().sum()

0

In [111]:
# Convert 'Bathrooms' column to integer
data['Bathrooms'] = data['Bathrooms'].astype(int)
data

,Price,Bathrooms,Car Parks,Property Type,Furnishing,type,size,unit,area,No_of_Bedrooms,No_of_servant_rooms
0,1250000,3,2,Serviced Residence,Fully Furnished,Built-up,1335.0,sq.,KLCC,2,1
1,1030000,4,2,Condominium (Corner),Partly Furnished,Built-up,1875.0,sq.,Dutamas,3,0
2,900000,3,2,Condominium (Corner),Partly Furnished,Built-up,1513.0,sq.,Bukit Jalil,4,1
3,5350000,5,4,Bungalow,Partly Furnished,Land area,7200.0,sq.,Taman Tun Dr Ismail,4,2
4,2600000,4,4,Semi-detached House,Partly Furnished,Land area,3600.0,sq.,Taman Tun Dr Ismail,5,0
...,...,...,...,...,...,...,...,...,...,...,...
28923,750000,2,1,Condominium,Partly Furnished,Built-up,915.0,sq.,Seputeh,3,0
28924,1400000,3,2,Condominium (Corner),Fully Furnished,Land area,1544.0,sq.,KL Sentral,3,1
28925,880000,1,1,Condominium (Corner),Partly Furnished,Built-up,650.0,sq.,KL Eco City,1,0
28926,2700000,6,3,Condominium (Corner),Partly Furnished,Built-up,3973.0,sq.,Sri Hartamas,4,2


In [112]:
# Convert 'Car Parks' column to integer
data['Car Parks'] = data['Car Parks'].astype(int)
data

,Price,Bathrooms,Car Parks,Property Type,Furnishing,type,size,unit,area,No_of_Bedrooms,No_of_servant_rooms
0,1250000,3,2,Serviced Residence,Fully Furnished,Built-up,1335.0,sq.,KLCC,2,1
1,1030000,4,2,Condominium (Corner),Partly Furnished,Built-up,1875.0,sq.,Dutamas,3,0
2,900000,3,2,Condominium (Corner),Partly Furnished,Built-up,1513.0,sq.,Bukit Jalil,4,1
3,5350000,5,4,Bungalow,Partly Furnished,Land area,7200.0,sq.,Taman Tun Dr Ismail,4,2
4,2600000,4,4,Semi-detached House,Partly Furnished,Land area,3600.0,sq.,Taman Tun Dr Ismail,5,0
...,...,...,...,...,...,...,...,...,...,...,...
28923,750000,2,1,Condominium,Partly Furnished,Built-up,915.0,sq.,Seputeh,3,0
28924,1400000,3,2,Condominium (Corner),Fully Furnished,Land area,1544.0,sq.,KL Sentral,3,1
28925,880000,1,1,Condominium (Corner),Partly Furnished,Built-up,650.0,sq.,KL Eco City,1,0
28926,2700000,6,3,Condominium (Corner),Partly Furnished,Built-up,3973.0,sq.,Sri Hartamas,4,2


In [113]:
data

,Price,Bathrooms,Car Parks,Property Type,Furnishing,type,size,unit,area,No_of_Bedrooms,No_of_servant_rooms
0,1250000,3,2,Serviced Residence,Fully Furnished,Built-up,1335.0,sq.,KLCC,2,1
1,1030000,4,2,Condominium (Corner),Partly Furnished,Built-up,1875.0,sq.,Dutamas,3,0
2,900000,3,2,Condominium (Corner),Partly Furnished,Built-up,1513.0,sq.,Bukit Jalil,4,1
3,5350000,5,4,Bungalow,Partly Furnished,Land area,7200.0,sq.,Taman Tun Dr Ismail,4,2
4,2600000,4,4,Semi-detached House,Partly Furnished,Land area,3600.0,sq.,Taman Tun Dr Ismail,5,0
...,...,...,...,...,...,...,...,...,...,...,...
28923,750000,2,1,Condominium,Partly Furnished,Built-up,915.0,sq.,Seputeh,3,0
28924,1400000,3,2,Condominium (Corner),Fully Furnished,Land area,1544.0,sq.,KL Sentral,3,1
28925,880000,1,1,Condominium (Corner),Partly Furnished,Built-up,650.0,sq.,KL Eco City,1,0
28926,2700000,6,3,Condominium (Corner),Partly Furnished,Built-up,3973.0,sq.,Sri Hartamas,4,2


In [114]:
data.isna().sum()

Price                  0
Bathrooms              0
Car Parks              0
Property Type          0
Furnishing             0
type                   0
size                   0
unit                   0
area                   0
No_of_Bedrooms         0
No_of_servant_rooms    0
dtype: int64

In [115]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28928 entries, 0 to 28927
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Price                28928 non-null  int32  
 1   Bathrooms            28928 non-null  int32  
 2   Car Parks            28928 non-null  int32  
 3   Property Type        28928 non-null  object 
 4   Furnishing           28928 non-null  object 
 5   type                 28928 non-null  object 
 6   size                 28928 non-null  float64
 7   unit                 28928 non-null  object 
 8   area                 28928 non-null  object 
 9   No_of_Bedrooms       28928 non-null  object 
 10  No_of_servant_rooms  28928 non-null  object 
dtypes: float64(1), int32(3), object(7)
memory usage: 2.1+ MB


In [116]:
data= data.dropna()
data.reset_index(drop=True, inplace=True) 
data

,Price,Bathrooms,Car Parks,Property Type,Furnishing,type,size,unit,area,No_of_Bedrooms,No_of_servant_rooms
0,1250000,3,2,Serviced Residence,Fully Furnished,Built-up,1335.0,sq.,KLCC,2,1
1,1030000,4,2,Condominium (Corner),Partly Furnished,Built-up,1875.0,sq.,Dutamas,3,0
2,900000,3,2,Condominium (Corner),Partly Furnished,Built-up,1513.0,sq.,Bukit Jalil,4,1
3,5350000,5,4,Bungalow,Partly Furnished,Land area,7200.0,sq.,Taman Tun Dr Ismail,4,2
4,2600000,4,4,Semi-detached House,Partly Furnished,Land area,3600.0,sq.,Taman Tun Dr Ismail,5,0
...,...,...,...,...,...,...,...,...,...,...,...
28923,750000,2,1,Condominium,Partly Furnished,Built-up,915.0,sq.,Seputeh,3,0
28924,1400000,3,2,Condominium (Corner),Fully Furnished,Land area,1544.0,sq.,KL Sentral,3,1
28925,880000,1,1,Condominium (Corner),Partly Furnished,Built-up,650.0,sq.,KL Eco City,1,0
28926,2700000,6,3,Condominium (Corner),Partly Furnished,Built-up,3973.0,sq.,Sri Hartamas,4,2


data['size'] = data['size'].astype(int)

In [117]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28928 entries, 0 to 28927
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Price                28928 non-null  int32  
 1   Bathrooms            28928 non-null  int32  
 2   Car Parks            28928 non-null  int32  
 3   Property Type        28928 non-null  object 
 4   Furnishing           28928 non-null  object 
 5   type                 28928 non-null  object 
 6   size                 28928 non-null  float64
 7   unit                 28928 non-null  object 
 8   area                 28928 non-null  object 
 9   No_of_Bedrooms       28928 non-null  object 
 10  No_of_servant_rooms  28928 non-null  object 
dtypes: float64(1), int32(3), object(7)
memory usage: 2.1+ MB


In [118]:
# Use regex to remove non-digit characters and keep only numbers
data['No_of_servant_rooms'] = data['No_of_servant_rooms'].str.replace(r'\D+', '', regex=True)

# Replace empty strings or NaN values with a placeholder value (e.g., '0')
data['No_of_servant_rooms'].replace('', '0', inplace=True)
data['No_of_servant_rooms'].fillna('0', inplace=True)

# Convert the column to integer type
data['No_of_servant_rooms'] = data['No_of_servant_rooms'].astype(int)


In [119]:
data

,Price,Bathrooms,Car Parks,Property Type,Furnishing,type,size,unit,area,No_of_Bedrooms,No_of_servant_rooms
0,1250000,3,2,Serviced Residence,Fully Furnished,Built-up,1335.0,sq.,KLCC,2,1
1,1030000,4,2,Condominium (Corner),Partly Furnished,Built-up,1875.0,sq.,Dutamas,3,0
2,900000,3,2,Condominium (Corner),Partly Furnished,Built-up,1513.0,sq.,Bukit Jalil,4,1
3,5350000,5,4,Bungalow,Partly Furnished,Land area,7200.0,sq.,Taman Tun Dr Ismail,4,2
4,2600000,4,4,Semi-detached House,Partly Furnished,Land area,3600.0,sq.,Taman Tun Dr Ismail,5,0
...,...,...,...,...,...,...,...,...,...,...,...
28923,750000,2,1,Condominium,Partly Furnished,Built-up,915.0,sq.,Seputeh,3,0
28924,1400000,3,2,Condominium (Corner),Fully Furnished,Land area,1544.0,sq.,KL Sentral,3,1
28925,880000,1,1,Condominium (Corner),Partly Furnished,Built-up,650.0,sq.,KL Eco City,1,0
28926,2700000,6,3,Condominium (Corner),Partly Furnished,Built-up,3973.0,sq.,Sri Hartamas,4,2


In [120]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28928 entries, 0 to 28927
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Price                28928 non-null  int32  
 1   Bathrooms            28928 non-null  int32  
 2   Car Parks            28928 non-null  int32  
 3   Property Type        28928 non-null  object 
 4   Furnishing           28928 non-null  object 
 5   type                 28928 non-null  object 
 6   size                 28928 non-null  float64
 7   unit                 28928 non-null  object 
 8   area                 28928 non-null  object 
 9   No_of_Bedrooms       28928 non-null  object 
 10  No_of_servant_rooms  28928 non-null  int32  
dtypes: float64(1), int32(4), object(6)
memory usage: 2.0+ MB


In [121]:
# Calculate minimum and maximum size values for studio records
min_size_studio = data.loc[data['No_of_Bedrooms'] == 'Studio', 'size'].min()
max_size_studio = data.loc[data['No_of_Bedrooms'] == 'Studio', 'size'].max()
mean_size_studio = data.loc[data['No_of_Bedrooms'] == 'Studio', 'size'].mean()

print("Minimum size for studio records:", min_size_studio)
print("Maximum size for studio records:", max_size_studio)
print("Mean size for studio records:", mean_size_studio)

Minimum size for studio records: 250.0
Maximum size for studio records: 243900.0
Mean size for studio records: 1423.7371794871794


In [122]:
data['No_of_Bedrooms'].value_counts()

No_of_Bedrooms
3           13928
4            4962
2            4488
5            2086
1            1990
6             748
Studio        312
7             255
8              97
9              22
10             20
12              5
20 Above        4
13              4
11              3
14              2
15              1
16              1
Name: count, dtype: int64

In [123]:
# Create a new column 'studio_apartment'
data['studio_apartment'] = data['No_of_Bedrooms'].apply(lambda x: 1 if x == 'Studio' else 0)

In [124]:
# Replace 'studio' values in 'No_of_Bedrooms' column with 0
data['No_of_Bedrooms'].replace({'Studio': 0, '20 Above': 21}, inplace=True)

data

,Price,Bathrooms,Car Parks,Property Type,Furnishing,type,size,unit,area,No_of_Bedrooms,No_of_servant_rooms,studio_apartment
0,1250000,3,2,Serviced Residence,Fully Furnished,Built-up,1335.0,sq.,KLCC,2,1,0
1,1030000,4,2,Condominium (Corner),Partly Furnished,Built-up,1875.0,sq.,Dutamas,3,0,0
2,900000,3,2,Condominium (Corner),Partly Furnished,Built-up,1513.0,sq.,Bukit Jalil,4,1,0
3,5350000,5,4,Bungalow,Partly Furnished,Land area,7200.0,sq.,Taman Tun Dr Ismail,4,2,0
4,2600000,4,4,Semi-detached House,Partly Furnished,Land area,3600.0,sq.,Taman Tun Dr Ismail,5,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
28923,750000,2,1,Condominium,Partly Furnished,Built-up,915.0,sq.,Seputeh,3,0,0
28924,1400000,3,2,Condominium (Corner),Fully Furnished,Land area,1544.0,sq.,KL Sentral,3,1,0
28925,880000,1,1,Condominium (Corner),Partly Furnished,Built-up,650.0,sq.,KL Eco City,1,0,0
28926,2700000,6,3,Condominium (Corner),Partly Furnished,Built-up,3973.0,sq.,Sri Hartamas,4,2,0


In [125]:
data.studio_apartment.value_counts()

studio_apartment
0    28616
1      312
Name: count, dtype: int64

In [126]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28928 entries, 0 to 28927
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Price                28928 non-null  int32  
 1   Bathrooms            28928 non-null  int32  
 2   Car Parks            28928 non-null  int32  
 3   Property Type        28928 non-null  object 
 4   Furnishing           28928 non-null  object 
 5   type                 28928 non-null  object 
 6   size                 28928 non-null  float64
 7   unit                 28928 non-null  object 
 8   area                 28928 non-null  object 
 9   No_of_Bedrooms       28928 non-null  object 
 10  No_of_servant_rooms  28928 non-null  int32  
 11  studio_apartment     28928 non-null  int64  
dtypes: float64(1), int32(4), int64(1), object(6)
memory usage: 2.2+ MB


In [127]:
data['No_of_Bedrooms']=data['No_of_Bedrooms'].astype(int)

In [128]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28928 entries, 0 to 28927
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Price                28928 non-null  int32  
 1   Bathrooms            28928 non-null  int32  
 2   Car Parks            28928 non-null  int32  
 3   Property Type        28928 non-null  object 
 4   Furnishing           28928 non-null  object 
 5   type                 28928 non-null  object 
 6   size                 28928 non-null  float64
 7   unit                 28928 non-null  object 
 8   area                 28928 non-null  object 
 9   No_of_Bedrooms       28928 non-null  int32  
 10  No_of_servant_rooms  28928 non-null  int32  
 11  studio_apartment     28928 non-null  int64  
dtypes: float64(1), int32(5), int64(1), object(5)
memory usage: 2.1+ MB


In [129]:
data

,Price,Bathrooms,Car Parks,Property Type,Furnishing,type,size,unit,area,No_of_Bedrooms,No_of_servant_rooms,studio_apartment
0,1250000,3,2,Serviced Residence,Fully Furnished,Built-up,1335.0,sq.,KLCC,2,1,0
1,1030000,4,2,Condominium (Corner),Partly Furnished,Built-up,1875.0,sq.,Dutamas,3,0,0
2,900000,3,2,Condominium (Corner),Partly Furnished,Built-up,1513.0,sq.,Bukit Jalil,4,1,0
3,5350000,5,4,Bungalow,Partly Furnished,Land area,7200.0,sq.,Taman Tun Dr Ismail,4,2,0
4,2600000,4,4,Semi-detached House,Partly Furnished,Land area,3600.0,sq.,Taman Tun Dr Ismail,5,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
28923,750000,2,1,Condominium,Partly Furnished,Built-up,915.0,sq.,Seputeh,3,0,0
28924,1400000,3,2,Condominium (Corner),Fully Furnished,Land area,1544.0,sq.,KL Sentral,3,1,0
28925,880000,1,1,Condominium (Corner),Partly Furnished,Built-up,650.0,sq.,KL Eco City,1,0,0
28926,2700000,6,3,Condominium (Corner),Partly Furnished,Built-up,3973.0,sq.,Sri Hartamas,4,2,0


In [130]:
def simplify_property_type(prop_type):
    super_types = [
        "Terrace/Link House", "Serviced Residence", "Condominium",
        "Semi-detached House", "Bungalow", "Apartment", "Townhouse",
        "Flat", "Residential Land", "Cluster House"
    ]
    for super_type in super_types:
        if re.search(super_type, prop_type, flags=re.IGNORECASE) is not None:
            return super_type
    return prop_type

# Assuming properties is your DataFrame containing the "Property Type" column
data["Property Type Supergroup"] = data["Property Type"].apply(simplify_property_type)
data.drop(columns='Property Type', inplace= True)

# Display value counts for 'Property Type Supergroup'
print(data["Property Type Supergroup"].value_counts(dropna=False))

Property Type Supergroup
Condominium            14487
Serviced Residence      7605
Terrace/Link House      3152
Bungalow                1409
Apartment               1037
Semi-detached House      823
Townhouse                284
Flat                     102
Residential Land          23
Cluster House              6
Name: count, dtype: int64


In [131]:
data.drop(columns= 'unit', inplace= True)

In [132]:
data

,Price,Bathrooms,Car Parks,Furnishing,type,size,area,No_of_Bedrooms,No_of_servant_rooms,studio_apartment,Property Type Supergroup
0,1250000,3,2,Fully Furnished,Built-up,1335.0,KLCC,2,1,0,Serviced Residence
1,1030000,4,2,Partly Furnished,Built-up,1875.0,Dutamas,3,0,0,Condominium
2,900000,3,2,Partly Furnished,Built-up,1513.0,Bukit Jalil,4,1,0,Condominium
3,5350000,5,4,Partly Furnished,Land area,7200.0,Taman Tun Dr Ismail,4,2,0,Bungalow
4,2600000,4,4,Partly Furnished,Land area,3600.0,Taman Tun Dr Ismail,5,0,0,Semi-detached House
...,...,...,...,...,...,...,...,...,...,...,...
28923,750000,2,1,Partly Furnished,Built-up,915.0,Seputeh,3,0,0,Condominium
28924,1400000,3,2,Fully Furnished,Land area,1544.0,KL Sentral,3,1,0,Condominium
28925,880000,1,1,Partly Furnished,Built-up,650.0,KL Eco City,1,0,0,Condominium
28926,2700000,6,3,Partly Furnished,Built-up,3973.0,Sri Hartamas,4,2,0,Condominium


In [133]:
data.to_csv('KL_clean_data.csv',index=False)

In [135]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28928 entries, 0 to 28927
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Price                     28928 non-null  int32  
 1   Bathrooms                 28928 non-null  int32  
 2   Car Parks                 28928 non-null  int32  
 3   Furnishing                28928 non-null  object 
 4   type                      28928 non-null  object 
 5   size                      28928 non-null  float64
 6   area                      28928 non-null  object 
 7   No_of_Bedrooms            28928 non-null  int32  
 8   No_of_servant_rooms       28928 non-null  int32  
 9   studio_apartment          28928 non-null  int64  
 10  Property Type Supergroup  28928 non-null  object 
dtypes: float64(1), int32(5), int64(1), object(4)
memory usage: 1.9+ MB
